In [4]:
!pip install selenium

In [5]:
# 키워드 검색
def insta_searching(word):
    url = 'https://www.instagram.com/explore/tags/'+word
    return url

In [6]:
# 첫 번째 게시물 클릭
def select_first(driver):
    first = driver.find_element_by_css_selector("div._9AhH0")
    first.click()
    time.sleep(3)
    
#select_first(driver)

In [7]:
# 게시물 정보 가져오기
# 본문 내용, 작성 일시, 위치 정보

import re
from bs4 import BeautifulSoup
import unicodedata

def get_content(driver):
    # html 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    
    # 본문 내용 가져오기
    try:
        content = soup.select('div.C4VMK>span')[0].text
        content = unicodedata.normalize('NFC', content)
    except:
        content = ''
    
    # 본문의 해시태그 가져오기
    tags= re.findall(r'#[^\s,\\]+', content)
    
    # 작성 일자 정보 가져오기
    date = soup.select('time.FH9sR.Nzb55')[0]['title']
    
    # 위치 정보 가져오기
    try:
        place = soup.select('div.JF9hh')[0].text
        place = unicodedata.normalize('NFC', place)
    except:
        place = ''
        
    # 수집한 정보 저장
    data = [content, date, place, tags]
    
    return data

#get_content(driver)

In [8]:
# 다음 게시물 열기
def move_next(driver):
    right = driver.find_element_by_css_selector('a.coreSpriteRightPaginationArrow')
    right.click()
    time.sleep(3)
    
#move_next(driver)

In [9]:
# 인스타그램 로그인
def instagram_login(id, pw):
    input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
    input_id.clear()
    input_id.send_keys(id)

    input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
    input_pw.clear()
    input_pw.send_keys(pw)
    input_pw.submit()
    time.sleep(3)
    
    driver.find_element_by_css_selector('.sqdOP.yWX7d.y3zKF').click()   #로그인정보 저장 '나중에 하기' 버튼 클릭

In [10]:
# 게시물 정보 가져오기
# 닉네임, 작성 일시, 위치 정보, 해시태그

import re
from bs4 import BeautifulSoup
import unicodedata

def get_data(driver):
    # html 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    
    # 본문 내용 가져오기
    try:
        content = soup.select('div.C4VMK>span')[0].text
        content = unicodedata.normalize('NFC', content)
    except:
        content = ''
    
    # 본문의 해시태그 가져오기
    tags= re.findall(r'#[^\s,\\]+', content)
    
    # 작성 일자 정보 가져오기
    date = soup.select('time.FH9sR.Nzb55')[0]['title']
    
    # 위치 정보 가져오기
    try:
        place = soup.select('div.JF9hh')[0].text
        place = unicodedata.normalize('NFC', place)
    except:
        place = ''
        
    # 이름(사용자 이름) 가져오기
    name = driver.find_element_by_css_selector(".sqdOP.yWX7d._8A5w5.ZIAjV").text
        
    
    # 수집한 정보 저장
    data = [name, date, place, tags]
    
    return data

#get_data(driver)

NameError: name 'driver' is not defined

In [11]:
# 여러 게시글 정보 수집하기

from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re

# 1. 크롬 브라우저 열기 및 로그인
driver = webdriver.Chrome('c:/users/mindo/like_lion/oneday/chromedriver.exe')
site = 'https://www.instagram.com/'
driver.get(site)
time.sleep(2)

email = 'mindori1019@gmail.com'
password = 'alswn4249!'
instagram_login(email, password)

# 2. 키워드 검색
key_word = '원데이클래스'
url = insta_searching(key_word)

# 3. 검색 페이지 접속
driver.get(url)
time.sleep(3)

# 4. 첫 번째 게시글 열기
select_first(driver)

# 5. 여러 게시물 수집하기
results =[]

target = 500  # 크롤링할 게시글 수
for i in range(target):
    try:
        data = get_data(driver)
        results.append(data)
        move_next(driver)
    except:
        time.sleep(2)
        move_next(driver)


In [ ]:
print(results[:2])

In [12]:
# 엑셀에 데이터 저장
import pandas as pd

results_df = pd.DataFrame(results)
results_df.columns =  ['name', 'date', 'place', 'tags']
results_df.to_excel('c:/users/mindo/like_lion/crawling_instagram.xlsx', index = False)